# 2D t-SNE for MFCC
### from 15sec-from-the-middle audio dataset

#### Visualization using vega and altair

pip install vega

pip install altair vega_datasets

### Importing

In [ ]:
%matplotlib inline
import glob
import numpy as np
import librosa
import os
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import pandas as pd
import vega
import altair as alt
import pandas as pd
import scipy.signal
import scipy.fftpack as fft
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

### Read audio files

In [ ]:
#File reading
all_dirs = []
for root, dirs, files in os.walk('./Dataset'): #change directory here
        for name in files:
            if '.wav' in name:
                filedir = os.path.join(root, name)
                all_dirs.append(filedir)
file_no = len(all_dirs)

### Compute MFCC

In [ ]:
#Feature Computation
all_mfcc = []
file_names = []
all_mean = []
print(file_no)
for i in range(file_no):
    if (librosa.get_duration(filename=all_dirs[i]) == 15.):
        file_names.append(all_dirs[i])
        
        #Load file
        y, sr = librosa.core.load(all_dirs[i], duration=15.)
        #Features
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        all_mfcc.append(mfcc.flatten())
        
        S = librosa.core.stft(y=y)
        S_db = librosa.core.power_to_db(np.abs(S)**2)
        all_mean.append(np.mean(S_db))

    sys.stdout.write("\rLoading %i recordings." % (i))
    sys.stdout.flush()

#Standardization
scl1 = StandardScaler()
all_mfcc_scaled = scl1.fit_transform(all_mfcc)

### t-SNE

In [ ]:
#TSNE
all_mfcc_scaled_red2 = TSNE(n_components=2).fit_transform(all_mfcc_scaled)

### Kmeans

In [ ]:
#KMeans
kmeans = KMeans(n_clusters=7, random_state=None).fit(all_mfcc_scaled_red2) #change number of clusters here
clusters = kmeans.predict(all_mfcc_scaled_red2)

### Collection labels

In [ ]:
#Collection-based
collection = []
for i in range(len(file_names)):
    if 'ArabMashriq' in file_names[i]:
        collection.append(0)
    elif 'EisenbergCollection' in file_names[i]:
        collection.append(1)
        
#x and y
mfcc1 = []
mfcc2 = []
for i in range(len(all_mfcc_scaled_red2)):
    mfcc1.append(all_mfcc_scaled_red2[i][0])
    mfcc2.append(all_mfcc_scaled_red2[i][1])

### Visualization

In [ ]:
#Visualization
df1 = pd.DataFrame({'x': mfcc1, 'y': mfcc2, 'color': clusters, 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})
chart1 = alt.Chart(df1).mark_circle(opacity=0.6, size=50).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()

df2 = pd.DataFrame({'x': mfcc1, 'y': mfcc2, 'color': np.asarray(all_mean), 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})
chart2 = alt.Chart(df2).mark_circle(opacity=0.6, size=50).encode(x='x', y='y', color='color:Q', href='path', tooltip=['filename']).interactive()

df3 = pd.DataFrame({'x': mfcc1, 'y': mfcc2, 'color': clusters, 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})
chart3 = alt.Chart(df3).mark_circle(size=80).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()
df4 = pd.DataFrame({'x': mfcc1, 'y': mfcc2, 'color': np.asarray(all_mean), 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})
chart4 = alt.Chart(df4).mark_circle(size=30).encode(x='x', y='y', color='color:Q', href='path', tooltip=['filename']).interactive()

#Collections
df5 = pd.DataFrame({'x': mfcc1, 'y': mfcc2, 'color': collection, 'path': np.asarray(file_names), 'filename': np.asarray(file_names)})
chart5 = alt.Chart(df5).mark_circle(opacity=0.6, size=50).encode(x='x', y='y', color='color:N', href='path', tooltip=['filename']).interactive()

display(chart1)
display(chart2)
display(chart3 + chart4)
display(chart5)